In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from flask import Flask, jsonify

In [12]:
db_path = "hawaii.sqlite"


In [14]:
engine = create_engine(f'sqlite:///{db_path}')

Base = automap_base()
Base.prepare(engine, reflect=True)

In [15]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [16]:
app = Flask(__name__)

In [17]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Precipitation: /api/v1.0/precipitation<br/>"
        f"List of Stations: /api/v1.0/stations<br/>"
        f"Temperature for one year: /api/v1.0/tobs<br/>"
        f"Temperature stat from the start date(yyyy-mm-dd): /api/v1.0/yyyy-mm-dd<br/>"
        f"Temperature stat from start to end dates(yyyy-mm-dd): /api/v1.0/yyyy-mm-dd/yyyy-mm-dd"
    )

In [18]:
@app.route('/api/v1.0/<start>')
def get_t_start(start):
    filter(Measurement.date >= start).all()
session = Session(engine)
queryresult = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
session.close()

In [20]:
print(queryresult)

None


In [10]:
 tobsall = []
for min,avg,max in queryresult:
    tobs_dict = {}
    tobs_dict["Min"] = min
    tobs_dict["Average"] = avg
    tobs_dict["Max"] = max
    tobsall.append(tobs_dict)

return jsonify(tobsall)

TypeError: 'NoneType' object is not iterable

In [ ]:
@app.route('/api/v1.0/<start>/<stop>')
def get_t_start_stop(start,stop):
    filter(Measurement.date >= start).filter(Measurement.date <= stop).all()
session = Session(engine)
queryresult = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        
session.close()

In [ ]:
 tobsall = []
for min,avg,max in queryresult:
    tobs_dict = {}
    tobs_dict["Min"] = min
    tobs_dict["Average"] = avg
    tobs_dict["Max"] = max
    tobsall.append(tobs_dict)

return jsonify(tobsall)

In [ ]:
@app.route('/api/v1.0/tobs')
def tobs():
    session = Session(engine)
    lateststr = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    latestdate = dt.datetime.strptime(lateststr, '%Y-%m-%d')
    querydate = dt.date(latestdate.year -1, latestdate.month, latestdate.day)
    sel = [Measurement.date,Measurement.tobs]
    queryresult = session.query(*sel).filter(Measurement.date >= querydate).all()
    session.close()

In [ ]:
tobsall = []
for date, tobs in queryresult:
    tobs_dict = {}
    tobs_dict["Date"] = date
    tobs_dict["Tobs"] = tobs
    tobsall.append(tobs_dict)
return jsonify(tobsall)

In [ ]:
@app.route('/api/v1.0/stations')
def stations():
    session = Session(engine)
    sel = [Station.station,Station.name,Station.latitude,Station.longitude,Station.elevation]
    queryresult = session.query(*sel).all()
    session.close()

In [ ]:
stations = []
for station,name,lat,lon,el in queryresult:
    station_dict = {}
    station_dict["Station"] = station
    station_dict["Name"] = name
    station_dict["Lat"] = lat
    station_dict["Lon"] = lon
    station_dict["Elevation"] = el
    stations.append(station_dict)

return jsonify(stations)

In [ ]:
@app.route('/api/v1.0/precipitation')
def precipitation():
    session = Session(engine)
    sel = [Measurement.date,Measurement.prcp]
    queryresult = session.query(*sel).all()
    session.close()

In [ ]:
precipitation = []
for date, prcp in queryresult:
    prcp_dict = {}
    prcp_dict["Date"] = date
    prcp_dict["Precipitation"] = prcp
    precipitation.append(prcp_dict)

return jsonify(precipitation)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)